In [5]:
from bs4 import BeautifulSoup
import requests
import ssl
from lxml import etree
import pandas as pd

ssl._create_default_https_context = ssl._create_unverified_context


#town=1 :臺南市中西區

url = 'https://okgo.tw/buty/town.html?county=15&town=1'   # country代表城市 / town代表區域
userAgent = "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.82 Safari/537.36"
headers = {
    'user-agent':userAgent
}



res = requests.get(url=url, headers=headers)

html = etree.HTML(res.text)

areaName = html.xpath('//*[@id="form1"]/div[3]/div[2]/div/div[3]/div[2]/div[4]/h2/text()')
#print(areaName)



#每個景點的連結
spotLink = html.xpath('//*[@id="form1"]/div[3]/div[2]/div/div[3]/div[2]/div[4]/ul/li/a[2]/@href')

linkList=[]

for i in spotLink:
    link = 'https://okgo.tw/' +i[2:]  
    #print(link)
    linkList.append(link)

    
#每個景點的名字
spotName = html.xpath('//*[@id="form1"]/div[3]/div[2]/div/div[3]/div[2]/div[4]/ul/li/a[2]/h3/text()')

for n in spotName:
    name = n          
    #print(name)
    



In [15]:
linkList[0:10]

['https://okgo.tw//butyview.html?id=1130',
 'https://okgo.tw//butyview.html?id=3095',
 'https://okgo.tw//butyview.html?id=3182',
 'https://okgo.tw//butyview.html?id=1131',
 'https://okgo.tw//butyview.html?id=3199',
 'https://okgo.tw//butyview.html?id=3286',
 'https://okgo.tw//butyview.html?id=1146',
 'https://okgo.tw//butyview.html?id=1760',
 'https://okgo.tw//butyview.html?id=3197',
 'https://okgo.tw//butyview.html?id=1132']

In [10]:
#articleUrl = link
userAgent = "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.82 Safari/537.36"
headers = {
    'user-agent':userAgent
}

spotInfo = []

for l in linkList:
    articleRes = requests.get(url=l, headers=headers)

    articleHtml = etree.HTML(articleRes.text)

    #地點
    s_name = articleHtml.xpath('//*[@id="form1"]/div[3]/div[2]/div/div[3]/div[1]/div[1]/h2/text()')
    
    
    #電話
    tel = articleHtml.xpath('//*[@id="form1"]/div[3]/div[2]/div/div[3]/div[1]/div[1]/text()')
    #print("電話：\n")
    #print(tel[4])

    #地址
    adress = articleHtml.xpath('//*[@id="form1"]/div[3]/div[2]/div/div[3]/div[1]/div[1]/text()[2]')
    #print('地址：\n')
    #print(tel[5])

    #文章內容
    content = articleHtml.xpath('//*[@id="form1"]/div[3]/div[2]/div/div[3]/div[1]/div[1]/div[3]/text()')
    #print('文章內容：\n')
    #print(content)

    #交通資訊
    trans = articleHtml.xpath('//*[@id="Buty_View_Traffic"]/div/text()')
    #print('交通資訊：\n')
    #print(trans[:-2])

    #開放時間
    time = articleHtml.xpath('//*[@id="Buty_View_Traffic"]/div[2]/text()')
    #print('開放時間：\n')
    #print(trans[-2:])

    data = {
        '景點':s_name[0],
        '電話':tel[4],
        '地址':tel[5],
        '文章內容':''.join(content),
        '交通資訊':''.join(trans[:-2]),
        '開放時間':''.join(trans[4:6])
    }

    spotInfo.append(data)



In [12]:
df = pd.DataFrame(spotInfo)
df.head()

,景點,電話,地址,文章內容,交通資訊,開放時間
0,赤崁樓,電話：06-2205647,地址：台南市民族路2段212號(民族路與赤崁街交岔口),赤崁樓位於今日的市民族路二段212號，原稱普羅民遮城，因漢人稱荷蘭人為紅毛，也叫做「番仔樓」...,大眾運輸:自台南火車站前搭17路市車，於赤崁樓下車即抵。自行開車：台南(仁德)交流道下，循1...,白天：AM08:30至PM05:30夜間：PM05:30至PM09:00
1,神農街（北勢街）,電話：-,地址：台南市中西區神農街,「神農街」昔日被稱為北勢街，為目前市保存最完善的老街。五條港在清初為台灣對外的主要門戶，而「...,國道一號：下永康出口交流道，往台南→中正北路／台1線→中正南路→小北路→中正南路→西門路四段...,
2,台南移民署,電話：-,地址：台南市中西區府前路2段370號,「移民署」因其獨特的歐式城堡外觀，搭配希臘風格的廊柱和精緻的藝術雕花，被封為全台最美公家機關...,,
3,台南孔廟,電話：06-2289013,地址：台南市南門路二號,孔子廟又稱文廟，是全台最早的文廟，清末前是的官辦學府，故有「全台首學」之稱，地點位於承天府鬼...,大眾運輸:自台南火車站前搭17、18路市公車，於建興國中站下車，循南門路北行可抵。自行開車：...,AM8：00至PM5：30(清明、端午、中秋三節不開放)
4,林百貨,電話：06-2213000,地址：台南市中西區忠義路二段63號,,,
